In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer

import warnings 
import gc
import os

import tokenizers
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaConfig

In [2]:
pd.plotting.register_matplotlib_converters()
pd.options.display.max_rows=50
pd.options.display.max_columns=100
plt.xkcd()
warnings.filterwarnings('ignore')

%matplotlib inline
sns.set_style('darkgrid')
plt.rcParams.update({'font.size':18, 
                    'font.family':'Humor Sans'})

In [3]:
train_df_main=pd.read_csv('data/train.csv').dropna()
test_df_main=pd.read_csv('data/test.csv')
submit=pd.read_csv('data/sample_submission.csv')

In [4]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [5]:
class TweetData(Dataset):
    def __init__(self,df,max_len=96):
        self.df=df
        self.max_len=max_len
        self.label='selected_text' in df
        roberta_files='roberta-base/'
        self.tokenizer = tokenizers.ByteLevelBPETokenizer(
            vocab_file = f'{roberta_files}vocab.json',
            merges_file = f'{roberta_files}merges.txt',
            lowercase=True,
            add_prefix_space=True)
        
    def __getitem__(self, idx):
        data={}
        row=self.df.iloc[idx]
        
        ids, masks, tweet, offsets = self.get_input_data(row)
        data['ids']=ids
        data['masks']=masks
        data['tweet']=tweet
        data['offsets']=offsets
        
        if self.label:
            start_idx, end_idx = self.get_target_idx(row,tweet,offsets)
            data['start_idx']=start_idx
            data['end_idx']=end_idx
        
        return data
    
    def __len__(self):
        return len(self.df)
    
    def get_input_data(self,row):
        tweet = " " + " ".join(row.text.lower().split())
        encoding = self.tokenizer.encode(tweet)
        sentiment_id = self.tokenizer.encode(row.sentiment).ids
        ids = [0] + sentiment_id + [2,2] + encoding.ids + [2]
        offsets = [(0,0)]*4 + encoding.offsets + [(0,0)]
        
        pad_len = self.max_len - len(ids)
        if pad_len>0:
            ids+=[1]*pad_len
            offsets += [(0,0)] * pad_len
            
        ids = torch.tensor(ids)
        masks = torch.where(ids!=1, torch.tensor(1), torch.tensor(0))
        offsets = torch.tensor(offsets)
        
        return ids, masks, tweet, offsets    
    
    def get_target_idx(self, row, tweet, offsets):
        selected_text=" "+" ".join(row.selected_text.lower().split())
        len_st = len(selected_text)-1
        idx0 = None
        idx1 = None
        
        for ind in (i for i,e in enumerate(tweet) if e==selected_text[1]):
            if " " + tweet[ind: ind+len_st] == selected_text:
                idx0 = ind
                idx1 = ind + len_st - 1
                break
            
        char_targets = [0]*len(tweet)
        if idx0!=None and idx1!=None:
            for ct in range(idx0, idx1+1):
                char_targets[ct]=1
                
        target_idx = []
        for j,(offset1,offset2) in enumerate(offsets):
            if sum(char_targets[offset1:offset2]) > 0:
                target_idx.append(j)
                
        start_idx = target_idx[0]
        end_idx = target_idx[-1]
        
        return start_idx, end_idx

In [6]:
tokenizer = tokenizers.ByteLevelBPETokenizer(
    vocab_file = 'roberta-base/vocab.json',
    merges_file = 'roberta-base/merges.txt',
    lowercase=True,
    add_prefix_space=True)

In [10]:
row=train_df_main.iloc[1007]
tweet=" "+" ".join(row.text.lower().split())
encoding=tokenizer.encode(tweet)
encoding

Encoding(num_tokens=28, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [11]:
[i for i in zip(encoding.tokens, encoding.ids)]

[('Ġlaughing', 11339),
 ('Ġfor', 13),
 ('Ġno', 117),
 ('Ġreason', 1219),
 ('...', 734),
 ('maybe', 31501),
 ('Ġits', 63),
 ('Ġbecause', 142),
 ('Ġits', 63),
 ('Ġ2', 132),
 (':', 35),
 ('27', 2518),
 ('Ġand', 8),
 ('Ġi', 939),
 ('`', 12905),
 ('m', 119),
 ('Ġtired', 7428),
 (',', 6),
 ('Ġhaha', 46116),
 ('.', 4),
 ('Ġmaybe', 2085),
 ('Ġi', 939),
 ('Ġshould', 197),
 ('Ġgo', 213),
 ('Ġto', 7),
 ('Ġbed', 3267),
 ('Ġnight', 363),
 ('!', 328)]

In [12]:
tweet

' laughing for no reason...maybe its because its 2:27 and i`m tired, haha. maybe i should go to bed night!'

In [13]:
sentiment_id=tokenizer.encode(row.sentiment).ids
ids=[0] + sentiment_id + [2,2] + encoding.ids + [2]
offsets=[(0,0)]*4 + encoding.offsets + [(0,0)]
pad_len = 96 - len(ids)
if(pad_len>0): 
    ids+=[1]*pad_len
    offsets+=[(0,0)]*pad_len
    
ids=torch.tensor(ids)
masks=torch.where(ids != 1, torch.tensor(1), torch.tensor(0))
offsets = torch.tensor(offsets)

In [14]:
# row=train_df.iloc[1007]
selected_text=" "+" ".join(row.selected_text.lower().split())
selected_text, row.text

(' i`m tired,',
 'Laughing for no reason...maybe its because its 2:27 and I`m tired, haha. Maybe i should go to bed  Night!')

In [15]:
idx0 = None
idx1 = None
len_st=len(selected_text)-1

for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
    if " " + tweet[ind: ind+len_st] == selected_text:
        idx0 = ind
        idx1 = ind + len_st - 1
        break
        
idx0, idx1

(57, 66)

In [16]:
char_targets = [0] * len(tweet)
if idx0 != None and idx1 != None:
    for ct in range(idx0, idx1 + 1):
        char_targets[ct] = 1
        
target_idx=[]
for j,(offset1,offset2) in enumerate(offsets):
    if sum(char_targets[offset1:offset2])>0:
        target_idx.append(j)
        
target_idx[0], target_idx[-1]

(17, 21)

In [25]:
def get_train_val_loaders(df, train_idx, val_idx, batch_size=8):
    train_df=df.iloc[train_idx]
    val_df=df.iloc[val_idx]
    
    train_loader=DataLoader(
        TweetData(train_df),
        batch_size=batch_size,
        shuffle=True,
        num_workers=0,
        drop_last=True)
    
    val_loader=DataLoader(
        TweetData(val_df),
        batch_size=batch_size,
        shuffle=False,
        num_workers=0)
    
    dl_dict = {'train': train_loader, 'val':val_loader}
    return dl_dict

def get_testloader(df, batch_size=32):
    loader = DataLoader(
        TweetDataset(df),
        batch_size=batch_size,
        shuffle=False,
        num_workers=2)
    return loader

In [18]:
class TweetNet(nn.Module):
    def __init__(self):
        super().__init__()
        config=RobertaConfig.from_pretrained(
            'roberta-base/config.json', output_hidden_states=True)
        self.roberta = RobertaModel.from_pretrained(
            'roberta-base/pytorch_model.bin', config=config)
        self.dropout=nn.Dropout(0.5)
        self.fc=nn.Linear(config.hidden_size, 2)
        
        nn.init.normal_(self.fc.weight, std=0.02)
        nn.init.normal_(self.fc.bias, 0)
        
    def forward(self, input_ids, attention_mask):
        _,_, hs = self.roberta(input_ids, attention_mask)
        x = torch.stack([hs[-1], hs[-2], hs[-3], hs[-4]])
        x = torch.mean(x,0)
        x = self.dropout(x)
        x = self.fc(x)
        start_logits, end_logits = x.split(1, dim=-1)
        start_logits=start_logits.squeeze(-1)
        end_logits=end_logits.squeeze(-1)
        
        return start_logits, end_logits

In [19]:
def loss_fn(start_logits, end_logits, start_positions, end_positions):
    ce_loss=nn.CrossEntropyLoss()
    start_loss=ce_loss(start_logits, start_positions)
    end_loss=ce_loss(end_logits, end_positions)
    total_loss=start_loss+end_loss
    return total_loss

In [20]:
def get_selected_text(text, start_idx, end_idx, offsets):
    selected_text = ""
    for ix in range(start_idx, end_idx + 1):
        selected_text += text[offsets[ix][0]: offsets[ix][1]]
        if (ix + 1) < len(offsets) and offsets[ix][1] < offsets[ix + 1][0]:
            selected_text += " "
    return selected_text

def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

def Jaccard_Score(text, start_idx, end_idx, start_logits, end_logits, offsets):
    start_pred = np.argmax(start_logits)
    end_pred = np.argmax(end_logits)
    if start_pred > end_pred:
        pred = text
    else:
        pred = get_selected_text(text, start_pred, end_pred, offsets)
        
    true = get_selected_text(text, start_idx, end_idx, offsets)
    
    return jaccard(true, pred)

In [21]:
def train_model(model, dl_dict, criterion, optimizer, n_epochs, filename):
    model.cuda()
    for epoch in range(n_epochs):
        for phase in ['train','val']:
            if phase=='train':
                model.train()
            else:
                model.eval()
                
            epoch_loss=0.0
            epoch_jaccard=0.0
            
            for data in (dl_dict[phase]):
                torch.cuda.empty_cache()
                ids = data['ids'].cuda()
                masks = data['masks'].cuda()
                tweet = data['tweet']
                offsets = data['offsets'].numpy()
                start_idx = data['start_idx'].cuda()
                end_idx = data['end_idx'].cuda()                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase=='train'):
                    start_logits, end_logits = model(ids, masks)
                    loss = criterion(start_logits, end_logits, start_idx, end_idx)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    epoch_loss += loss.item() * len(ids)
                    
                    start_idx = start_idx.cpu().detach().numpy()
                    end_idx = end_idx.cpu().detach().numpy()
                    start_logits = torch.softmax(start_logits, dim=1).cpu().detach().numpy()
                    end_logits = torch.softmax(end_logits, dim=1).cpu().detach().numpy()
                    
                    for i in range(len(ids)):
                        jaccard_score = Jaccard_Score(tweet[i], start_idx[i], end_idx[i], 
                                                     start_logits[i], end_logits[i], offsets[i])
                        epoch_jaccard+=jaccard_score
                        
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_jaccard = epoch_jaccard/len(dataloaders_dict[phase].dataset)
            
            print('Epoch {}/{} | {:^5} | Loss: {:.4f} | Jaccard: {:.4f}'.format(
                        epoch+1, n_epochs, phase, epoch_loss, epoch_jaccard))
            
    torch.save(model.state_dict(), filename)

In [28]:
n_epochs=3
batch_size=32
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=69)

In [29]:
%%time

train_df_main['text']=train_df_main['text'].astype(str)
train_df_main['selected_text'] = train_df_main['selected_text'].astype(str)

for fold, (train_idx, val_idx) in enumerate(skf.split(train_df_main, train_df_main.sentiment), start = 1):
    print(f'Fold: {fold}')
    
    model = TweetNet()
    optimizer = optim.AdamW(model.parameters(),lr = 3e-5, betas = (0.85, 0.995))
    criterion = loss_fn
    dataloaders_dict = get_train_val_loaders(train_df_main, train_idx, val_idx, batch_size)
    
    train_model(model, dataloaders_dict, criterion, optimizer,
                    n_epochs, f'roberta_fold{fold}.pth')

Fold: 1


RuntimeError: CUDA out of memory. Tried to allocate 36.00 MiB (GPU 0; 4.00 GiB total capacity; 2.82 GiB already allocated; 22.77 MiB free; 3.02 GiB reserved in total by PyTorch)

In [37]:
data=next(iter(dataloaders_dict['train']))
data['end_idx']

tensor([17,  7, 34, 14, 30,  7,  8, 19, 10, 27, 16, 20,  8,  8, 23, 36, 28,  7,
        26, 34, 17, 23, 10, 13, 41, 12,  6, 12, 22, 14, 27, 12])